In [1]:
!pip install folium -q

In [88]:
import geopandas as gpd
import numpy as np
import pandas as pd
from IPython.display import display
import folium
import branca.colormap as cm

In [17]:
bra_municipios = gpd.read_file("zip://BR_Municipios_2021.zip")

simplified_bra_municipios = bra_municipios.copy()
simplified_bra_municipios['geometry'] = simplified_bra_municipios['geometry'].simplify(tolerance = 0.01)

simplified_bra_municipios.head()

,CD_MUN,NM_MUN,SIGLA,AREA_KM2,geometry
0,1100015,Alta Floresta D'Oeste,RO,7067.127,"POLYGON ((-62.19465 -11.82746, -62.17004 -11.8..."
1,1100023,Ariquemes,RO,4426.571,"POLYGON ((-62.53648 -9.73222, -62.49559 -9.772..."
2,1100031,Cabixi,RO,1314.352,"POLYGON ((-60.37119 -13.36655, -60.38792 -13.4..."
3,1100049,Cacoal,RO,3793.000,"POLYGON ((-61.0008 -11.29737, -61.00302 -11.40..."
4,1100056,Cerejeiras,RO,2783.300,"POLYGON ((-61.49976 -13.00525, -61.43437 -13.0..."


In [26]:
simplified_bra_municipios['CD_MUN'] = simplified_bra_municipios['CD_MUN'].astype(str)

In [95]:
bra_castana = pd.read_csv("bra_castana_production_by_municipality.csv")
bra_castana['CD_MUN'] = bra_castana['CD_MUN'].astype(str)
bra_castana =bra_castana.fillna(0)
bra_castana.dtypes

CD_MUN          object
Municipality    object
2018             int64
2019             int64
2020             int64
2021             int64
2022             int64
dtype: object

In [100]:
bra_gdf_castana = simplified_bra_municipios.merge(bra_castana, how = 'left', left_on = 'CD_MUN', right_on = 'CD_MUN')
bra_gdf_castana = bra_gdf_castana.fillna(0)
bra_gdf_castana.head()

,CD_MUN,NM_MUN,SIGLA,AREA_KM2,geometry,Municipality,2018,2019,2020,2021,2022
0,1100015,Alta Floresta D'Oeste,RO,7067.127,"POLYGON ((-62.19465 -11.82746, -62.17004 -11.8...",Alta Floresta D'Oeste (RO),5.0,6.0,5.0,5.0,5.0
1,1100023,Ariquemes,RO,4426.571,"POLYGON ((-62.53648 -9.73222, -62.49559 -9.772...",Ariquemes (RO),0.0,0.0,0.0,0.0,0.0
2,1100031,Cabixi,RO,1314.352,"POLYGON ((-60.37119 -13.36655, -60.38792 -13.4...",Cabixi (RO),3.0,3.0,3.0,3.0,4.0
3,1100049,Cacoal,RO,3793.000,"POLYGON ((-61.0008 -11.29737, -61.00302 -11.40...",Cacoal (RO),9.0,8.0,7.0,9.0,9.0
4,1100056,Cerejeiras,RO,2783.300,"POLYGON ((-61.49976 -13.00525, -61.43437 -13.0...",Cerejeiras (RO),0.0,0.0,0.0,0.0,0.0


In [101]:
def get_colormap(values) -> cm.ColorMap:
    min_value, max_value = min(values), max(values)
    colormap = cm.linear.YlOrRd_09.scale(min_value, max_value)
    return colormap

castana_colormap = get_colormap(bra_gdf_castana['2022'])

In [102]:
m = folium.Map(location=[-3.4653, -62.2159], zoom_start=5)

def get_color(value):
    if value == None:
        return '#fff5f0'
    if value >= 0 and value <= 1:
        return '#fff5f0'
    if value > 1 and value <= 200:
        return '#fdccb8'
    if value > 200 and value <= 1000:
        return '#fc8f6f'
    if value > 1000 and value <= 2000:
        return '#f44d37'
    if value > 2000 and value <= 3000:
        return '#c5161b'
    if value > 3000:
        return '#67000d'
    else:
        return '#fff5f0'


brazil_castana_2018_layer = folium.GeoJson(
    bra_gdf_castana,
    name = '2018 Castana Production in Brazil',
    show = False,
    style_function = lambda feature: {
        'fillColor': castana_colormap(feature['properties']['2018']),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.75
    },
    highlight_function=lambda feature: {
        'fillColor': 'yellow',   # Change color on hover
        'color': 'black',         # Keep border black on hover
        'weight': 1.5,            # Increase weight on hover
        'fillOpacity': 0.7
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['NM_MUN', '2018'],  # Replace with the name of your column
        aliases=['Municipality:', '2018 Brazil Nut Production'],             # Tooltip alias
        localize=True                   # Localize numbers
    )
).add_to(m)

brazil_castana_2019_layer = folium.GeoJson(
    bra_gdf_castana,
    show = False,
    name = '2019 Castana Production in Brazil',
    style_function = lambda feature: {
        'fillColor': castana_colormap(feature['properties']['2019']),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.75
    },
    highlight_function=lambda feature: {
        'fillColor': 'yellow',   # Change color on hover
        'color': 'black',         # Keep border black on hover
        'weight': 1.5,            # Increase weight on hover
        'fillOpacity': 0.7
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['NM_MUN', '2019'],  # Replace with the name of your column
        aliases=['Municipality:', '2019 Brazil Nut Production'],             # Tooltip alias
        localize=True                   # Localize numbers
    )
).add_to(m)

brazil_castana_2020_layer = folium.GeoJson(
    bra_gdf_castana,
    show = False,
    name = '2020 Castana Production in Brazil',
    style_function = lambda feature: {
        'fillColor': castana_colormap(feature['properties']['2020']),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.75
    },
    highlight_function=lambda feature: {
        'fillColor': 'yellow',   # Change color on hover
        'color': 'black',         # Keep border black on hover
        'weight': 1.5,            # Increase weight on hover
        'fillOpacity': 0.7
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['NM_MUN', '2020'],  # Replace with the name of your column
        aliases=['Municipality:', '2020 Brazil Nut Production'],             # Tooltip alias
        localize=True                   # Localize numbers
    )
).add_to(m)

brazil_castana_2021_layer = folium.GeoJson(
    bra_gdf_castana,
    show = False,
    name = '2021 Castana Production in Brazil',
    style_function = lambda feature: {
        'fillColor': castana_colormap(feature['properties']['2021']),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.75
    },
    highlight_function=lambda feature: {
        'fillColor': 'yellow',   # Change color on hover
        'color': 'black',         # Keep border black on hover
        'weight': 1.5,            # Increase weight on hover
        'fillOpacity': 0.7
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['NM_MUN', '2021'],  # Replace with the name of your column
        aliases=['Municipality:', '2021 Brazil Nut Production'],             # Tooltip alias
        localize=True                   # Localize numbers
    )
).add_to(m)

brazil_castana_2022_layer = folium.GeoJson(
    bra_gdf_castana,
    name = '2022 Castana Production in Brazil',
    style_function = lambda feature: {
        'fillColor': castana_colormap(feature['properties']['2022']),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.75
    },
    highlight_function=lambda feature: {
        'fillColor': 'yellow',   # Change color on hover
        'color': 'black',         # Keep border black on hover
        'weight': 1.5,            # Increase weight on hover
        'fillOpacity': 0.7
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['NM_MUN', '2022'],  # Replace with the name of your column
        aliases=['Municipality:', '2022 Brazil Nut Production'],             # Tooltip alias
        localize=True                   # Localize numbers
    )
).add_to(m)



folium.LayerControl(collapsed = False).add_to(m)


In [ ]:
# gdf.to_file("brazil_municipios.geojson", driver = "geojson")